In [1]:
import numpy as np
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Model, Reaction, Metabolite
from os.path import join

In [2]:
data_dir = "/Users/phivri/Documents/GitHub/Biosustain/MoGeoModel"

In [3]:
Cljung_Model = cobra.io.read_sbml_model(join(data_dir,'Models/C-ljungdahlii_iHN637.xml'))

medium = Cljung_Model.medium
medium['EX_h2_e'] = 1000
medium['EX_h2o_e'] = 1000
medium['EX_k_e'] = 0
medium['EX_fru_e'] = 0
medium['EX_mg2_e'] = 0
medium['EX_mn2_e'] = 0
medium['EX_na1_e'] = 0
medium['EX_nh4_e'] = 0
medium['EX_ni2_e'] = 0
medium['EX_pi_e'] = 0
medium['EX_mobd_e'] = 0
medium['EX_zn2_e'] = 0
medium['EX_ribflv_e'] = 0
medium['EX_btn_e'] = 0
medium['EX_so4_e'] = 0
medium['EX_thm_e'] = 0
medium['EX_ca2_e'] = 0
medium['EX_cl_e'] = 0
medium['EX_co2_e'] = 10
medium['EX_cobalt2_e'] = 0
medium['EX_cu2_e'] = 0
medium['EX_fe2_e'] = 0
medium['EX_fol_e'] = 0
medium['EX_h_e'] = 1000
Cljung_Model.medium = medium

Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


In [4]:
Cljung_Model.reactions.ATPM.bounds = 0, 1000

Cljung_Model.reactions.ACt2r.knock_out()

acT = Reaction("ACt")
acT.name = "Acetate transporter"
acT.add_metabolites({ 
    Cljung_Model.metabolites.ac_c: -1.0,
    Cljung_Model.metabolites.ac_e: 1.0
})

mthfd2 = Reaction("MTHFD2")
mthfd2.name = "Methylenetetrahydrofolate dehydrogenase (NAD)"

mthfd2.add_metabolites({
    Cljung_Model.metabolites.mlthf_c: -1.0,
    Cljung_Model.metabolites.nad_c: -1.0,
    Cljung_Model.metabolites.methf_c: 1.0,
    Cljung_Model.metabolites.nadh_c: 1.0,
})
print(mthfd2.check_mass_balance())    #ensure correct mass balance
print(mthfd2.bounds)                  #Ensure correct bounds
print(mthfd2.reaction)

Cljung_Model.add_reactions([acT, mthfd2])
Cljung_Model.reactions.MTHFD2.bounds = -1000.0, 1000.0


{}
(0.0, 1000.0)
mlthf_c + nad_c --> methf_c + nadh_c


In [5]:
Cljung_Model.reactions.ACACT1r.bounds = 0,1000.0
Cljung_Model.reactions.HACD1.bounds = 0,1000.0
Cljung_Model.reactions.ECOAH1.bounds = 0,1000.0
Cljung_Model.reactions.ACOAD1z.bounds = 0,1000.0
Cljung_Model.reactions.PBUTT.bounds = 0,1000.0
Cljung_Model.reactions.BUTKr.bounds = -1000.0,0
Cljung_Model.reactions.BTCOARx.bounds = 0,1000.0
Cljung_Model.reactions.ALCD4.bounds = -1000.0,0
Cljung_Model.reactions.BTOHt.bounds = -1000.0, 1000.0
Cljung_Model.reactions.BUTt.bounds = -1000.0, 1000.0
Cljung_Model.reactions.HYDFDN.bounds = -1000.0, 1000.0

# Find core reactions manually

In [6]:
#Cljung_Model.objective = "ATPM"
#Cljung_Model.objective = "EX_ac_e"
Cljung_Model.objective = "EX_etoh_e"
#Cljung_Model.objective = "EX_but_e"
#Cljung_Model.objective = "EX_btoh_e"
#Cljung_Model.objective = "EX_btd_RR_e"
#print(Cljung_Model.slim_optimize())
#solution = Cljung_Model.optimize()
#Cljung_Model.pfba()
#solution.fluxes[solution.fluxes != 0].index

solution = cobra.flux_analysis.pfba(Cljung_Model)
solution.fluxes[solution.fluxes != 0].index

Index(['EX_h2_e', 'EX_h2o_e', 'EX_co2_e', 'EX_etoh_e', 'ACKr', 'CO2t',
       'ALCD2y', 'ACAFDOR', 'CODH_ACS', 'ETOHt', 'H2Ot', 'H2td', 'FDH7',
       'FRNDPR2r', 'FTHFLi', 'METR', 'MTHFD', 'MTHFC', 'MTHFR5', 'PTAr',
       'HYDFDN'],
      dtype='object')

In [7]:
Cljung_Model.metabolites.but_c

Metabolite identifier,but_c
Name,Butyrate (n-C4:0)
Memory address,0x07fd4703aac50
Formula,C4H7O2
Compartment,c
In 2 reaction(s),"BUTt, BUTKr"


In [8]:
Cljung_Model.reactions.ATPS4r

Reaction identifier,ATPS4r
Name,ATP synthase (four protons for one ATP)
Memory address,0x07fd4728c4358
Stoichiometry,adp_c + 4.0 h_e + pi_c <=> atp_c + h2o_c + 3.0 h_c ADP C10H12N5O10P2 + 4.0 H+ + Phosphate <=> ATP C10H12N5O13P3 + H2O H2O + 3.0 H+
GPR,( CLJU_RS01160 and CLJU_RS01165 and CLJU_RS01170 ) and ( CLJU_RS01175 and CLJU_RS01180 and...
Lower bound,-1000.0
Upper bound,1000.0


In [9]:
WLmodel = Model('Wood_Ljungdahl_Core')

WLmodel.add_reactions([
#Transport & Exchange:
Cljung_Model.reactions.EX_h2_e,
Cljung_Model.reactions.EX_h2o_e,
Cljung_Model.reactions.EX_co2_e,
Cljung_Model.reactions.EX_co_e,
Cljung_Model.reactions.EX_pi_e,
Cljung_Model.reactions.EX_h_e,
Cljung_Model.reactions.EX_ac_e,
Cljung_Model.reactions.EX_etoh_e,
Cljung_Model.reactions.EX_btd_RR_e,
Cljung_Model.reactions.EX_but_e,
Cljung_Model.reactions.EX_btoh_e,
Cljung_Model.reactions.H2td,
Cljung_Model.reactions.H2Ot,
Cljung_Model.reactions.CO2t,
Cljung_Model.reactions.COt,
Cljung_Model.reactions.ACt,
Cljung_Model.reactions.ETOHt,
Cljung_Model.reactions.BTDt_RR,
Cljung_Model.reactions.BUTt,
Cljung_Model.reactions.BTOHt,

Cljung_Model.reactions.FDH7,
Cljung_Model.reactions.FTHFLi,
Cljung_Model.reactions.MTHFC,
Cljung_Model.reactions.MTHFD2,
Cljung_Model.reactions.MTHFR5,
Cljung_Model.reactions.METR,
Cljung_Model.reactions.CODH4,
Cljung_Model.reactions.CODH_ACS,
Cljung_Model.reactions.PTAr,
Cljung_Model.reactions.ACKr,
Cljung_Model.reactions.ACAFDOR,
Cljung_Model.reactions.ALCD2x,

Cljung_Model.reactions.HYDFDN,
Cljung_Model.reactions.RNF,
Cljung_Model.reactions.ATPS4r,
    
Cljung_Model.reactions.POR,
Cljung_Model.reactions.ACLS,
Cljung_Model.reactions.ACLDC,
Cljung_Model.reactions.BTDD_RR,
    
Cljung_Model.reactions.ACACT1r,
Cljung_Model.reactions.HACD1,
Cljung_Model.reactions.ECOAH1,
Cljung_Model.reactions.ACOAD1z,
Cljung_Model.reactions.BTCOARx,
Cljung_Model.reactions.ALCD4,
Cljung_Model.reactions.PBUTT,
Cljung_Model.reactions.BUTKr])

# Add Acetone Pathway

In [10]:
WLmodelActn = WLmodel

WLmodelActn.add_metabolites(Metabolite(id='acac_c',formula = 'C4H6O3', name = 'Acetoacetate', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_c',formula = 'C3H6O', name = 'Acetone', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_e', formula = 'C3H6O', name = 'Acetone', compartment = 'e'))

In [11]:
ctfAB_ac = Reaction("ctfAB_ac")
ctfAB_ac.name = "Acetate-acetoacetate CoA-transferase"

ctfAB_ac.add_metabolites({
    WLmodelActn.metabolites.ac_c: -1.0,      #Acetate
    WLmodelActn.metabolites.aacoa_c: -1.0,   #Acetoacetyl-CoA
    WLmodelActn.metabolites.acac_c: 1.0,     #Acetoacetate
    WLmodelActn.metabolites.accoa_c: 1.0,    #Acetyl-CoA
    WLmodelActn.metabolites.h_c: -1.0
})
print(ctfAB_ac.check_mass_balance())    #ensure correct mass balance
print(ctfAB_ac.bounds)                  #Ensure correct bounds
print(ctfAB_ac.reaction)                #Ensure correct reaction

{}
(0.0, 1000.0)
aacoa_c + ac_c + h_c --> acac_c + accoa_c


In [12]:
adc = Reaction("ADC")
adc.name = "acetoacetate decarboxylase"

adc.add_metabolites({
    WLmodelActn.metabolites.acac_c: -1.0,     #Acetoacetate
#    WLmodelActn.metabolites.h_c: -1.0, 
    WLmodelActn.metabolites.act_c: 1.0,
    WLmodelActn.metabolites.co2_c: 1.0
})
print(adc.check_mass_balance())    #ensure correct mass balance
print(adc.reaction)                #Ensure correct reaction

{}
acac_c --> act_c + co2_c


In [13]:
actT = Reaction("ACTt")
actT.name = "Acetone transporter"
actT.add_metabolites({ 
    WLmodelActn.metabolites.act_c: -1.0,
    WLmodelActn.metabolites.act_e: 1.0
})
actT

Reaction identifier,ACTt
Name,Acetone transporter
Memory address,0x07fd4725b4208
Stoichiometry,act_c --> act_e Acetone --> Acetone
GPR,
Lower bound,0.0
Upper bound,1000.0


In [14]:
act_ex = Reaction("EX_act_e")
act_ex.name = "Acetone exchange"
act_ex.add_metabolites({ 
    WLmodelActn.metabolites.act_e: -1.0,
})
act_ex

Reaction identifier,EX_act_e
Name,Acetone exchange
Memory address,0x07fd4725d7da0
Stoichiometry,act_e --> Acetone -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [15]:
WLmodelActn.add_reactions([ctfAB_ac, adc, actT, act_ex])

In [16]:
WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("atp_c"), type="sink")
WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("adp_c"), type="sink")
#WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("nadh_c"), type="sink")
#WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("nad_c"), type="sink")
#WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("fdxo_42_c"), type="sink")
#WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("fdxr_42_c"), type="sink")
#WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("coa_c"), type="sink")
#WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("h_c"), type="sink")
WLmodelActn.add_boundary(WLmodelActn.metabolites.get_by_id("pi_c"), type="sink")

Reaction identifier,SK_pi_c
Name,Phosphate sink
Memory address,0x07fd47259ceb8
Stoichiometry,pi_c <=> Phosphate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


Use this instead:
https://www.frontiersin.org/articles/10.3389/fbioe.2020.621166/full

In [17]:
WLmodelActn.objective = "EX_ac_e"

medium_carb = WLmodelActn.medium
medium_carb['EX_co_e'] = 10
medium_carb['EX_co2_e'] = 0
medium_carb['EX_h2_e'] = 0

WLmodelActn.medium = medium_carb
WLmodelActn.optimize()
WLmodelActn.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
co_e,EX_co_e,10,1,19.35%
h2o_e,EX_h2o_e,0.8333,0,0.00%
h_e,EX_h_e,1.667,0,0.00%
adp_c,SK_adp_c,4.167,10,80.65%
pi_c,SK_pi_c,4.167,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-2.5,2,9.68%
co2_e,EX_co2_e,-5,1,9.68%
atp_c,SK_atp_c,-4.167,10,80.65%


In [18]:
WLmodelActn.reactions.CODH_ACS.bounds = 1,1

In [19]:
WLmodelActn.objective = "EX_ac_e"
#WLmodelActn.objective = "EX_etoh_e"
#WLmodelActn.objective = "EX_act_e"
#WLmodelActn.objective = "EX_but_e"
#WLmodelActn.objective = "EX_btoh_e"
#WLmodelActn.objective = "EX_btd_RR_e"

medium_hom = WLmodelActn.medium
medium_hom['EX_co_e'] = 0
medium_hom['EX_co2_e'] = 10
medium_hom['EX_h2_e'] = 1000
WLmodelActn.medium = medium_hom
solution = WLmodelActn.optimize()

print("CO: ",WLmodelActn.reactions.EX_co_e.flux)
print("H2O: ",WLmodelActn.reactions.EX_h2o_e.flux)
print("CO2: ",WLmodelActn.reactions.EX_co2_e.flux)
print("H2: ",WLmodelActn.reactions.EX_h2_e.flux)
print("Product: ",WLmodelActn.slim_optimize())
print("ATP: ",WLmodelActn.reactions.SK_atp_c.flux)

CO:  0.0
H2O:  2.3333333333333335
CO2:  -2.0
H2:  -4.0
Product:  1.0
ATP:  0.3333333333333333


In [20]:
Builder(model = WLmodelActn, map_json = join(data_dir,'Models/EscherMaps/WL_core_Actn_SKATP.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'EX_h2_e': -4.0, 'EX_h2o_e': 2.3333333333333335, 'EX_co2_e': -2.0, 'EX_co_e': 0.0, 'EX_…

In [22]:
objectives = pd.Series(["EX_ac_e","EX_etoh_e","EX_act_e","EX_but_e","EX_btoh_e","EX_btd_RR_e"])
formulas = pd.Series(["CH3COOH","C2H5OH","C3H6O","C4H7O2","C4H9OH","C4H10O2"], index=objectives)
Cmol =  pd.Series([2,2,3,4,4,4], index=objectives)

for x in objectives:
    print(x)
    WLmodelActn.objective = x
    with WLmodelActn:
        medium_carb = WLmodelActn.medium
        medium_carb['EX_co_e'] = 10
        medium_carb['EX_co2_e'] = 0
        medium_carb['EX_h2_e'] = 0
        WLmodelActn.medium = medium_carb
        WLmodelActn.optimize()
        print(f" carb: {round(WLmodelActn.reactions.EX_co_e.flux,2)} CO + {round(WLmodelActn.reactions.EX_h2o_e.flux,2)} H2O + {round(WLmodelActn.reactions.EX_co2_e.flux,2)} CO2 + {round(WLmodelActn.slim_optimize(),2)} {formulas[x]} + {round(WLmodelActn.reactions.EX_h_e.flux,2)} H+ + {round(WLmodelActn.reactions.SK_atp_c.flux,2)} ATP")
        print("   ATP/Electron donor: ", round(-WLmodelActn.reactions.SK_atp_c.flux/WLmodelActn.reactions.EX_co_e.flux,2))
        print("   Product/Electron donor: ", round(-WLmodelActn.slim_optimize()*Cmol[x]/WLmodelActn.reactions.EX_co_e.flux,2))

    with WLmodelActn:
        medium_hom = WLmodelActn.medium
        medium_hom['EX_co_e'] = 0
        medium_hom['EX_co2_e'] = 10
        medium_hom['EX_h2_e'] = 1000
        WLmodelActn.medium = medium_hom
        WLmodelActn.optimize()
        print(f" hom: {round(WLmodelActn.reactions.EX_co2_e.flux,2)} CO2 + {round(WLmodelActn.reactions.EX_h2_e.flux,2)} H2 + {round(WLmodelActn.reactions.EX_h2o_e.flux,2)} H2O + {round(WLmodelActn.slim_optimize(),2)} {formulas[x]} + {round(WLmodelActn.reactions.EX_h_e.flux,2)} H+ + {round(WLmodelActn.reactions.SK_atp_c.flux,2)} ATP")
        print("   ATP/Electron donor: ", round(-WLmodelActn.reactions.SK_atp_c.flux/WLmodelActn.reactions.EX_h2_e.flux,2))
        print("   Product/Electron donor: ", round(-WLmodelActn.slim_optimize()*Cmol[x]/WLmodelActn.reactions.EX_h2_e.flux,2))

EX_ac_e
 carb: -4.0 CO + -0.33 H2O + 2.0 CO2 + 1.0 CH3COOH + -0.67 H+ + 1.67 ATP
   ATP/Electron donor:  0.42
   Product/Electron donor:  0.5
 hom: -2.0 CO2 + -4.0 H2 + 2.33 H2O + 1.0 CH3COOH + 0.67 H+ + 0.33 ATP
   ATP/Electron donor:  0.08
   Product/Electron donor:  0.5
EX_etoh_e
 carb: -6.0 CO + -0.33 H2O + 4.0 CO2 + 1.0 C2H5OH + -2.67 H+ + 2.67 ATP
   ATP/Electron donor:  0.44
   Product/Electron donor:  0.33
 hom: -2.0 CO2 + -6.0 H2 + 3.67 H2O + 1.0 C2H5OH + -0.67 H+ + 0.67 ATP
   ATP/Electron donor:  0.11
   Product/Electron donor:  0.33
EX_act_e
 carb: -4.0 CO + -0.17 H2O + 2.5 CO2 + 0.5 C3H6O + -1.33 H+ + 1.33 ATP
   ATP/Electron donor:  0.33
   Product/Electron donor:  0.38
 hom: -1.5 CO2 + -4.0 H2 + 2.5 H2O + 0.5 C3H6O + 0.0 H+ + 0.0 ATP
   ATP/Electron donor:  0.0
   Product/Electron donor:  0.38
EX_but_e
 carb: -5.0 CO + 0.17 H2O + 3.0 CO2 + 0.5 C4H7O2 + -1.67 H+ + 2.17 ATP
   ATP/Electron donor:  0.43
   Product/Electron donor:  0.4
 hom: -2.0 CO2 + -5.0 H2 + 3.5 H2O + 0.